In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.getcwd()

'/Users/chuqiwang/Documents/US_PFAS_Project/data_cleaning/Illinois'

In [3]:
il_df1 = pd.read_excel('Export_Scott AFB.xlsx')

In [4]:
il_df1

,Component ID,Component,State,Installation Name,Installation Type,Lab Report Id,Laboratory Sample ID,Sample Date,Analysis Method,CAS Number,Analyte Abbreviation,Final Result,Limit of Detection,Final Units,Final Qualification,Treatment System,Pre Or Post Treatment
0,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,757124-72-4,4:2FTS,NaN,8.0,ng/L,U,No,NaN
1,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,27619-97-2,6:2FTS,NaN,8.0,ng/L,U,No,NaN
2,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,39108-34-4,8:2FTS,NaN,8.0,ng/L,U,No,NaN
3,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,2991-50-6,NEtFOSAA,NaN,8.0,ng/L,U,No,NaN
4,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,2355-31-9,NMeFOSAA,NaN,8.0,ng/L,U,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,2706-90-3,PFPeA,NaN,4.0,ng/L,U,No,NaN
452,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,2706-91-4,PFPeS,NaN,4.0,ng/L,U,No,NaN
453,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,376-06-7,PFTeDA,NaN,4.0,ng/L,U,No,NaN
454,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,72629-94-8,PFTrDA,NaN,4.0,ng/L,U,No,NaN


In [5]:
columns = {
    'sample_id': str,
    'sampling_location': str,
    'data_source': str,
    'latitude': float,
    'longitude': float,
    'sampling_date': 'datetime64[ns]',
    'pws_id': str,
    'water_system_name': str,
    'population': int,
    'well_type': str,
    'well_depth': float,
    'units': str,
    'source_type': str,
    'state': str,
    'city': str,
    'county': str,
    'MRL': float,
    'DL': float,
    'analytical_methods': str,
    'is_detected': bool,
    'analyte': str,
    'concentration': float,
    'pfas_total': float,
    'note': str   
}

In [6]:
df1 = pd.DataFrame({col: pd.Series(dtype=dt) for col, dt in columns.items()})
df1['analyte'] = il_df1['Analyte Abbreviation'].str.upper()
df1['concentration'] = il_df1['Final Result'].fillna(0)
df1['sampling_location'] = il_df1['Installation Name']
df1['DL'] = il_df1['Limit of Detection']
df1['sampling_date'] = pd.to_datetime(il_df1['Sample Date'])
df1['sample_id'] = il_df1['Laboratory Sample ID']
df1['units'] = il_df1['Final Units']
df1['state'] = il_df1['State']
df1['analytical_methods'] = il_df1['Analysis Method']
df1['data_source'] = 'DoD'
df1['source_type'] = 'drinking water'

In [7]:
df1['is_detected'] = df1['concentration'] != 0

In [8]:
df = df1

In [9]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from geopy.geocoders import OpenCage

geolocator = OpenCage(api_key="c7f16120befe4e3fb85e7a082e620fdb")

def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return (address, location.latitude, location.longitude)
        else:
            return (address, None, None)
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return (address, None, None)

addresses_to_query = df.loc[df['latitude'].isna() | df['longitude'].isna(), 'sampling_location'].unique()

with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(geocode_address, addresses_to_query))

location_cache = {address: (lat, lon) for address, lat, lon in results}


In [10]:
location_cache

{'Scott AFB': (38.543839, -89.8527741)}

In [11]:
df.loc[df['latitude'].isna(), 'latitude'] = df.loc[df['latitude'].isna(), 'sampling_location'].map(
    lambda x: location_cache.get(x, (None, None))[0]
)

df.loc[df['longitude'].isna(), 'longitude'] = df.loc[df['longitude'].isna(), 'sampling_location'].map(
    lambda x: location_cache.get(x, (None, None))[1]
)


In [12]:
df

,sample_id,sampling_location,data_source,latitude,longitude,sampling_date,pws_id,water_system_name,population,well_type,...,city,county,MRL,DL,analytical_methods,is_detected,analyte,concentration,pfas_total,note
0,FA91647-1,Scott AFB,DoD,38.543839,-89.852774,2021-12-12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,QSM_B15,False,4:2FTS,0.0,NaN,NaN
1,FA91647-1,Scott AFB,DoD,38.543839,-89.852774,2021-12-12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,QSM_B15,False,6:2FTS,0.0,NaN,NaN
2,FA91647-1,Scott AFB,DoD,38.543839,-89.852774,2021-12-12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,QSM_B15,False,8:2FTS,0.0,NaN,NaN
3,FA91647-1,Scott AFB,DoD,38.543839,-89.852774,2021-12-12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,QSM_B15,False,NETFOSAA,0.0,NaN,NaN
4,FA91647-1,Scott AFB,DoD,38.543839,-89.852774,2021-12-12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,QSM_B15,False,NMEFOSAA,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,FC14593-2,Scott AFB,DoD,38.543839,-89.852774,2024-04-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,QSM_B15,False,PFPEA,0.0,NaN,NaN
452,FC14593-2,Scott AFB,DoD,38.543839,-89.852774,2024-04-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,QSM_B15,False,PFPES,0.0,NaN,NaN
453,FC14593-2,Scott AFB,DoD,38.543839,-89.852774,2024-04-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,QSM_B15,False,PFTEDA,0.0,NaN,NaN
454,FC14593-2,Scott AFB,DoD,38.543839,-89.852774,2024-04-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,QSM_B15,False,PFTRDA,0.0,NaN,NaN


In [13]:
df1.to_csv('illinois_dw.csv')